# Portfolio optimization using Python

In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import datetime as dt
from pandas_datareader import data as wb
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sn
import yfinance as yf
%matplotlib inline

In [2]:
# Importing financial data for a list of stocks from Yahoo Finance
# User inputs the stock symbols, e.g., "SBIN HDFC ONGC WIPRO HEROMOTOCO LT ITC COALINDIA RELIANCE NCC"
stocks = input("Enter stock symbols separated by spaces: ").split()
asset = [stock.upper() + '.NS' for stock in stocks]
asset.sort()

Enter stock symbols separated by spaces: SBIN HDFC  ONGC WIPRO HEROMOTOCO LT ITC COALINDIA RELIANCE NCC


In [3]:
# User specifies the number of years of data to analyze
years = float(input('Number of years of data you want to analyze: '))
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365 * years)

Number of years of data you want to analyze: 10


In [4]:
# Using yfinance to download historical stock price data
yf.pdr_override()
pf_data = wb.get_data_yahoo(asset, start=startDate, end=endDate)['Adj Close']

[*********************100%***********************]  10 of 10 completed


In [5]:
# Scaling the data and preparing it for LSTM prediction
# LSTM (Long Short-Term Memory) is a type of recurrent neural network often used for time series prediction.

def Scale_data_set(data):
    dataset = data.values
    training_data_len = int(np.ceil(len(dataset) * 0.95))
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)
    return training_data_len, scaled_data, dataset, scaler

In [6]:
# Creating training data for LSTM
def Creating_training_data(training_data_len, scaled_data):
    train_data = scaled_data[0:int(training_data_len), :]
    x_train = []
    y_train = []

    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    return x_train, y_train

In [7]:
# Building and training the LSTM model
def LSTM_model(x_train, y_train):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, batch_size=1, epochs=1)
    return model

In [8]:
# Predicting data using the trained LSTM model
def miscellaneous(model, training_data_len, scaled_data, dataset, scaler):
    test_data = scaled_data[training_data_len - 60:, :]
    x_test = []
    y_test = dataset[training_data_len:, :]

    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])

    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
    return predictions, rmse

In [9]:
# Plotting the chart with predicted values
def Ploting_Prediction(predictions, training_data_len, data, title):
    train = data[:training_data_len]
    valid = data[training_data_len:]
    valid['Predictions'] = predictions
    plt.figure(figsize=(16, 6))
    plt.title(title)
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Close Price in Rs', fontsize=18)
    plt.plot(train['Close'])
    plt.plot(valid[['Close', 'Predictions']])
    plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
    plt.show()
    return train, valid

In [10]:
# Plotting the graph using Plotly
def plot_graph_plotly(predictions, training_data_len, data, title):
    train = data[:training_data_len]
    valid = data[training_data_len:]
    valid['Predictions'] = predictions

    train_data = go.Scatter(
        name='Training data',
        x=data.index,
        y=train['Close'],
        marker=dict(color='blue', size=5,)
    )

    Val_data = go.Scatter(
        name='Actual Value',
        x=valid.index,
        y=valid['Close'],
        marker=dict(color='green', size=5,)
    )

    Prediction_data = go.Scatter(
        name='Predicted Data',
        x=valid.index,
        y=valid['Predictions'],
        marker=dict(color='red', size=5,)
    )

    data = [train_data, Val_data, Prediction_data]

    layout = go.Layout(
        title=title,
        yaxis=dict(title='Close Price in Rs'),
        xaxis=dict(title='Date'),
        showlegend=True,
        legend=dict(
            x=0.83, y=0, traceorder='normal',
            bgcolor='#E2E2E2',
            bordercolor='black',
            borderwidth=2),
        width=980,
        height=500)

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    return train, valid

In [11]:
# Compiling all functions into one
def Stock_prediction(data, title):
    training_data_len, scaled_data, dataset, scaler = Scale_data_set(data)
    x_train, y_train = Creating_training_data(training_data_len, scaled_data)
    model = LSTM_model(x_train, y_train)
    predictions, rmse = miscellaneous(model, training_data_len, scaled_data, dataset, scaler)
    train, valid = plot_graph_plotly(predictions, training_data_len, data, title)
    return train, valid, rmse, predictions

In [14]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential


# Sending Data for Prediction for each stock in the portfolio
data = {}
for st in asset:
    data['{}'.format(st)] = pf_data.filter([st])

for i in asset:
    DATA = data[i]
    DATA = DATA.rename(columns={i: "Close"})
    train, valid, rmse, predictions = Stock_prediction(DATA, i)
    print("'Root Mean Squared Error' of {} = ".format(i) + str(rmse))

NameError: name 'LSTM' is not defined

In [ ]:
# Picking the best-performing stocks based on annual returns
retrn = (pf_data / pf_data.shift(1)) - 1
annual_returns = retrn.mean() * 250 * 100
annual_returns.values[::-1].sort()
sorted_annual_return = annual_returns[0:5]
assets = list(sorted_annual_return.index)
pf_data = pf_data.filter(assets)
returns = retrn.filter(assets)
num_assets = len(assets)

In [ ]:
# Generating random portfolio weights and calculating portfolio statistics
weights = np.random.random(num_assets)
weights /= np.sum(weights)

mean_annual_returns = np.sum(weights * returns.mean()) *